# Connection à la base de données 'fcd'

In [1]:
import pandas as pd
import pymongo

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# Lecture dans la base

In [2]:
cursor = collection.find()
entries = list(cursor)

#entries[:]

df = pd.DataFrame(entries)
df.head()
print('\ndone')


done


# Entrée 1 : géo

In [30]:
lon = float(input("longitude -> "))
lat = float(input("latitude -> "))
rad = float(input("rayon (km) -> "))

print('\ndone')

longitude -> -73.88
latitude -> 45.21
rayon (km) -> 10

done


# Entrée 2 : date

In [32]:
from datetime import timedelta
from datetime import datetime

year = int(input("Annee -> "))
month = int(input("Mois -> "))
day = int(input("Jour -> "))
hour = int(input("Heure -> "))
minutes = int(input("Minutes -> "))
seconds = int(input("Secondes -> "))
date = datetime(year, month, day, hour, minutes, seconds)

print('\ndone')

Annee -> 2020
Mois -> 3
Jour -> 2
Heure -> 15
Minutes -> 50
Secondes -> 54

done


# Entrée 3 : période

In [35]:
T = int(input("Période (minutes) -> "))
delta = timedelta(minutes = float(T/2))

datemin = date - delta
datemax = date + delta

datemin = datetime.timestamp(datemin)
datemax = datetime.timestamp(datemax)

print('\ndone')

Période (minutes) -> 62

done


# QUERY 1 

In [38]:
import pprint


# QUERY 1
# Calculer la densité [nombre de points] sur une Zone géographique Z et une période T 

def density_list(lon, lat, rad, datemin, datemax, option, zone, periode, params):

    dico={}
    query01={}
    query02={}
    center=[-1,-1]
    
    # points dans une zone Z
    if zone:
        
        center=[lon,lat]
        
        query01 = {
            "location.geo": {
                "$nearSphere": {
                    "$geometry": {
                        "type": "Point",
                        "coordinates": [ lon, lat]
                    },
                    "$minDistance": 0,
                    "$maxDistance": rad*1000 # mètres
                }
            }
        }

    # points à une période T
    if periode :
        query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}


    # requête finale
    query = {'$and': [query01, query02]}

    if option != {} :
        query = {'$and': [query, option]}
        
    liste=list(collection.find(query).distinct("asset_id"))
    dico['neighbors_id']=liste
    dico['density']=len(liste)
    dico['period']=[datemin, datemax]
    z = {}
    z['center'] = center
    z['radius'] = rad
    dico['zone'] = z
    dico['settings'] = params
    return dico


now = datetime.now()
result = density_list(lon, lat, rad, datemin, datemax, {}, True, True, {'function': 'density_list', 'query_date' : now})
pprint.pprint(result)

print('\ndone')

{'density': 0,
 'neighbors_id': [],
 'period': [1583158794.0, 1583162514.0],
 'settings': {'function': 'density_list',
              'query_date': datetime.datetime(2020, 7, 28, 15, 49, 55, 977651)},
 'zone': {'center': [-73.88, 45.21], 'radius': 10.0}}
done


# QUERY 2

In [40]:
import pprint

# QUERY 2
# Retourner le nombre de points voisins d'un asset X dans une période T.

def time_gap(current, new):

    td = abs(current - new)
    res = td/60
    return res


def asset_density(id_asset, rad, datemin, datemax, params):

    # les points d'un asset X
    query01 = {"asset_id": id_asset}

    # points à une période T
    query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}

    # les points d'un asset X à une période T
    query03 = {'$and': [query01, query02]}

    tmp = list(collection.find(query03))

    lon = 0
    lat = 0
    lon_average = 0
    lat_average = 0
    
    density = 0
    res = {}
    liste = []
    num = 0
    time = 0
    
    for doc in tmp:
        
        if time_gap(time, doc['recorded_at']) > 2:
            
            time = doc['recorded_at']
            
            lon = doc['location']['geo']['coordinates'][0]
            lat = doc['location']['geo']['coordinates'][1]         
            lon_average += lon
            lat_average += lat
            num += 1
            
            d_list = density_list(lon, lat, rad, datemin, datemax,
                                  {"asset_id": {"$ne": id_asset}}, True, True, params)

            if d_list['density'] > 0:

                for neighbor in d_list['neighbors_id']:

                    if neighbor not in liste:
                        density += 1
                        liste.append(neighbor)

    if density > 0:
        lon_average = lon_average/num
        lat_average = lat_average/num
        
        res['asset_id'] = id_asset
        res['density'] = density
        res['neighbors_id']= liste
        
        z = {}
        z['center'] = [lon_average, lat_average]
        z['radius'] = rad
        res['zone'] = z
        res['period'] = [datemin, datemax]
        res['settings'] = params
        
    return res
  

now = datetime.now()
result = asset_density('14429000019112', rad, datemin, datemax, {'function': 'asset_density', 'query_date' : now})

pprint.pprint(result)

print('\ndone')

{'asset_id': '14429000019112',
 'density': 6,
 'neighbors_id': ['14429000008834',
                  '14429000012521',
                  '14429000015615',
                  '14429000034707',
                  '14429000040936',
                  '14429000060124'],
 'period': [1583158794.0, 1583162514.0],
 'settings': {'function': 'asset_density',
              'query_date': datetime.datetime(2020, 7, 28, 15, 56, 18, 397199)},
 'zone': {'center': [-113.403, 53.430144444444444], 'radius': 10.0}}

done


# QUERY 3

In [41]:
# QUERY 3
# Sur la base de tous les assets, retourner le Top – k [e.g. top 10] des zones les plus denses


def top_k_zones(k, rad, datemin, datemax):
    
    # list des id des différents asset
    ids_asset = list(collection.distinct("asset_id"))

    zones = []
    now = datetime.now()

    for id_ in ids_asset:

        zone = asset_density(id_, rad, datemin, datemax, {'function': 'top_k_zones', 'query_date' : now})

        if zone != {}:
            zones.append(zone)

    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


zones = top_k_zones(15, rad, datemin, datemax)

print("top", len(zones), "\n----------------------")
pprint.pprint(zones)

print('\ndone')

top 15 
----------------------
[{'asset_id': '14429000059761',
  'density': 26,
  'neighbors_id': ['14429000003116',
                   '14429000005848',
                   '14429000015714',
                   '14429000026877',
                   '14429000030150',
                   '14429000032073',
                   '14429000034814',
                   '14429000038856',
                   '14429000039524',
                   '14429000039532',
                   '14429000048111',
                   '14429000050190',
                   '14429000051404',
                   '14429000053855',
                   '14429000054135',
                   '14429000054879',
                   '14429000059837',
                   '14429000036553',
                   '14429000049796',
                   '14429000010277',
                   '14429000049531',
                   '14429000060694',
                   '14429000014790',
                   '14429000057039',
                   '144290000397

# Query4

In [47]:
import math
import datetime

# Query 4
# Retournez le top k des périodes/ zones les plus denses 


# in meters
def distance(lat1, lat2, lon1, lon2):
    R = 6372800  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    d = 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return d


def top_k_zones2(k, rad):
    
    now = datetime.datetime.now()
    
    # toutes les points de la base de données
    base = list(collection.find())
    
    # tri selon la longitude puis la latitude
    base = sorted(base, key=lambda point:point['location']['geo']['coordinates'])
    print('sorting done\n')
    
    
    zones = []
    lon = 0
    lat = 0
    nextP = False
    
    for point in base:
        
        if distance(lat, point['location']['geo']['coordinates'][1],
                    lon, point['location']['geo']['coordinates'][0]) >= rad*1000*2:
            
            if len(zones) > 0:
                for z in zones:
                     
                    if distance(z['zone']['center'][1], point['location']['geo']['coordinates'][1],
                                z['zone']['center'][0], point['location']['geo']['coordinates'][0]) < rad*1000*2:
                        nextP = True
                        break
            if nextP:
                nextP = False
                continue
                
            lon = point['location']['geo']['coordinates'][0]
            lat = point['location']['geo']['coordinates'][1]
            
            d = density_list(lon, lat, rad, 0, 0,{}, True, False, {'function': 'top_k_zones2', 'query_date' : now})
            if d['density'] > 0:
                zones.append(d)
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


def top_k_periods(k, delta, option, params):
    
    # toutes les points de la base de données
    base = list(collection.find())
    base = sorted(base, key=lambda point:point['recorded_at'])
    print('sorting done\n')
    
    
    zones = []
    
    # intervalle de temps en heures converti en secondes
    T = delta
    delta = datetime.timedelta(hours=delta).total_seconds()
    
    end = base[len(base)-1]['recorded_at']
    datemin = base[0]['recorded_at']
    datemax = datemin + delta
    
    
    while datemin < end:
        
        d = density_list(-1, -1, -1, datemin, datemax, option, False, True, params)
        
        if d['density'] > 0:
            zones.append(d)
             
        datemin = datemax
        datemax = datemin + delta
        
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if  k == 'all':
        return zones
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]



rad = 10
k = 10
delta = 6
now = datetime.datetime.now()

# top k des zones les plus denses
#zones = top_k_zones2(k, rad)

# top k des périodes les plus denses
zones = top_k_periods(k, delta, {}, {'function': 'top_k_periods', 'query_date' : now, 'delta': T})

pprint.pprint(zones)

print('\ndone')

sorting done

[{'density': 88,
  'neighbors_id': ['14256004530491',
                   '14256004535292',
                   '14256004569226',
                   '14256004574044',
                   '14256004592574',
                   '14256004664720',
                   '14295000006551',
                   '14429000003116',
                   '14429000004924',
                   '14429000005848',
                   '14429000006838',
                   '14429000008834',
                   '14429000008883',
                   '14429000008909',
                   '14429000010095',
                   '14429000012521',
                   '14429000014790',
                   '14429000015615',
                   '14429000015714',
                   '14429000016597',
                   '14429000019013',
                   '14429000019112',
                   '14429000022157',
                   '14429000023551',
                   '14429000026877',
                   '14429000029509',
       

In [28]:
def top_k_zones_periods(k, rad, delta):
    
    periods = top_k_periods('all', delta, {}, {})
    
    # toutes les points de la base de données
    base = list(collection.find())
    
    # tri selon la longitude puis la latitude
    base = sorted(base, key=lambda point:point['location']['geo']['coordinates'])
    print('sorting done\n')
    
    
    zones = []
    lon = 0
    lat = 0
    nextP = False
    
    for point in base:
        
        if distance(lat, point['location']['geo']['coordinates'][1],
                    lon, point['location']['geo']['coordinates'][0]) >= rad*1000*2:
            
            if len(zones) > 0:
                for z in zones:
                     
                    if distance(z['zone']['center'][1], point['location']['geo']['coordinates'][1],
                                z['zone']['center'][0], point['location']['geo']['coordinates'][0]) < rad*1000*2:
                        nextP = True
                        break
                        
            if nextP:
                nextP = False
                continue
                
            lon = point['location']['geo']['coordinates'][0]
            lat = point['location']['geo']['coordinates'][1]
            
            for p in periods:
                
                if p['density'] < 20:
                    break
                    
                zones.append(density_list(lon, lat, rad, p['period'][0], p['period'][1], {}, True, True))
                
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]




# top k des zones/périodes les plus denses
zones = top_k_zones_periods(k, rad, delta)

pprint.pprint(zones)
print('\ndone')

sorting done

sorting done

[{'density': 31,
  'neighbors_id': ['14429000003116',
                   '14429000005848',
                   '14429000008909',
                   '14429000010095',
                   '14429000014790',
                   '14429000015714',
                   '14429000016597',
                   '14429000030150',
                   '14429000032073',
                   '14429000034814',
                   '14429000035001',
                   '14429000035217',
                   '14429000038708',
                   '14429000038856',
                   '14429000039524',
                   '14429000039755',
                   '14429000048111',
                   '14429000049531',
                   '14429000049796',
                   '14429000050190',
                   '14429000050877',
                   '14429000051404',
                   '14429000053301',
                   '14429000053855',
                   '14429000054135',
                   '1442900005

# Query 5

In [48]:
# QUERY 5
# Retournez le top k des périodes T où nous avons le plus de voitures qui sont en circulation

# les trajets (sans stay points)
op = {"moving": 1}

zones = top_k_periods(k, delta, op, {'function': 'top_k_periods_moving', 'query_date' : now, 'delta': delta})
pprint.pprint(zones)

print('\ndone')

sorting done

[{'density': 86,
  'neighbors_id': ['14256004535292',
                   '14256004559268',
                   '14256004644482',
                   '14256004664720',
                   '14295000006551',
                   '14295000017319',
                   '14429000003116',
                   '14429000004924',
                   '14429000005012',
                   '14429000005848',
                   '14429000006838',
                   '14429000008834',
                   '14429000008883',
                   '14429000008909',
                   '14429000010095',
                   '14429000012521',
                   '14429000014790',
                   '14429000015615',
                   '14429000015714',
                   '14429000016597',
                   '14429000019013',
                   '14429000019112',
                   '14429000022157',
                   '14429000023551',
                   '14429000026877',
                   '14429000029509',
       